# 支持向量机 SVM

## 模型

有监督学习，找出超平面分界解决分类问题，最优超平面根据支持向量决定

## 实现

In [161]:
import numpy as np

#! 修改输入数据
# 行为对象，列为指标
datax = np.array([
    [1, 1.1, 1, 0],
    [2, 2.2, 0, 1],
    [3, 3.3, 1, 0],
    [4, 4.4, 0, 1],
    [5, 5.5, 0, 1],
    [6, 6.6, 1, 0],
    [9, 3.2, 1, 0],
    [8, 1.1, 1, 0],
])
datay = np.array(['f', 'm', 'm', 'm', 'f', 'm', 'm', 'm'])

#! 修改模型训练超参数
# 训练集比值
TRAIN_SET_RATIO = 0.8

### 拟合

In [162]:
from sklearn.svm import SVC # 回归则用 SVR
from sklearn.preprocessing import StandardScaler

# 标准化
std = StandardScaler()
x_std = std.fit_transform(datax)

train_size = int(x_std.shape[0] * TRAIN_SET_RATIO)
train_x, test_x = x_std[:train_size, :], x_std[train_size:, :]
train_y, test_y = datay[:train_size], datay[train_size:]

model = SVC(
    C=2,
    kernel='linear', # 通常选用线性核，更具解释性
)

model.fit(train_x, train_y)

print("模型根据训练集评估得分(分类时是评分，回归时是R^2): ", model.score(test_x, test_y))

模型根据训练集评估得分(分类时是评分，回归时是R^2):  1.0


### 模型选参

SVM 对参数敏感，需要谨慎选参

可以使用网格搜索进行自动选参，并进行交叉验证

网格搜索就是排列组合所有方案，选出最优

In [163]:
from sklearn.model_selection import GridSearchCV

#! 修改网格搜索方案选择
params = {
    'kernel': ('linear', 'rbf', 'poly'),
    'C': [0.01, 0.1, 0.5, 1, 2, 10, 100],
}

model_select = GridSearchCV(
    # 待选参模型
    model, 
    # 方案
    param_grid=params, 
    # 交叉验证折数
    cv=2,
    error_score='raise',
)
model_select.fit(datax, datay)

print(f"最优参数选择: {model_select.best_params_}")
print(f"最优方案得分: {model_select.best_score_}")

最优参数选择: {'C': 0.01, 'kernel': 'linear'}
最优方案得分: 0.75


### 预测

In [164]:
#
#! 修改预测数据
newx = np.array([
    [15, 2.1, 0, 1],
])

result = model.predict(newx)

print(f"预测结果: {result}")

预测结果: ['m']


### 解释性

只有当 `kernal='linear'` 时，SVM 才具有解释性

In [165]:
weights = model.coef_[0]

print(f"各指标在线性核 SVM 决策中的权重: {weights}")

各指标在线性核 SVM 决策中的权重: [ 1.36570226e-07  2.12587518e-07  6.78694772e-17 -6.78694772e-17]


当然使用非线性核时，也可以用 [事后解释性工具](../机器学习/解释性分析.ipynb) 进行解释